In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<pre>Interface for search functionality in the last cell</pre>

In [0]:
import sqlite3
# connect database
conn = sqlite3.connect('test.db')
print('Opened database successfully.');
c = conn.cursor()

Opened database successfully.


<pre>Create table schema</pre>

In [0]:
c.execute('''DROP TABLE IF EXISTS postings ''')
c.execute(''' 
        CREATE TABLE IF NOT EXISTS postings
       (word VARCHAR(100) PRIMARY KEY,
       postinglist_freq_tfidf TEXT );''')

print('Table postings created successfully.');

c.execute('''DROP TABLE IF EXISTS docmag ''')
c.execute(''' 
        CREATE TABLE IF NOT EXISTS docmag
       (docid INT PRIMARY KEY     NOT NULL,
       maxf INT     NOT NULL,
       mag FLOAT);''')
print('Table docmag created successfully.');

conn.commit()

Table postings created successfully.
Table docmag created successfully.


<pre>Insert into table docmag</pre>

In [0]:
# store data to database
import re
conn = sqlite3.connect('test.db')
print('Opened database successfully.');
c = conn.cursor()

# delete content in database

sql_delete = """ DELETE FROM docmag"""
c.execute(sql_delete)

# Inserting data from doc_mag
with open("/content/drive/My Drive/doc_mag.txt") as file1:
    f11 = file1.readlines()
    # print(f11)
for x in f11:
  
   x.strip()
   x = re.findall(r'\d+\.?\d*', x)
  #  print(x)
   sql = """insert into docmag(docid,maxf,mag) values (?,?,?)"""
   c.execute(sql,(str(x[0]),x[1],x[2]))
   conn.commit()



Opened database successfully.


<pre>Insert into table postings</pre>

In [0]:
sql_delete = """ DELETE FROM postings"""
c.execute(sql_delete)
# Inserting data from inv_txt
with open("/content/drive/My Drive/inv_idx.txt") as file2:
    f22 = file2.readlines()

for x in f22:
   y = re.findall(r'\'(.*)\'[^\[]*(\[.*\])', x)
   #print(y[0][0]) 
   sql_1 = "INSERT INTO postings VALUES (?,?)"
   c.execute(sql_1,[y[0][0],y[0][1]])
   conn.commit()

In [0]:
sql_look = """ SELECT * FROM docmag """
c.execute(sql_look)
print(c.fetchone())
sql_look = """ SELECT * FROM postings """
c.execute(sql_look)
print(c.fetchone())
#conn.close()

(17140, 6, 0.008196472745010674)
('mar', '[(8086, 1, 1.579030475288173e-05), (411, 1, 1.579030475288173e-05), (1399, 1, 1.3534546931041483e-05), (5155, 1, 1.579030475288173e-05), (4663, 1, 3.158060950576346e-05), (3931, 1, 7.895152376440865e-06), (6699, 1, 1.3534546931041483e-05), (3662, 1, 4.737091425864519e-05), (4277, 1, 8.612893501571853e-06), (1202, 1, 2.3685457129322596e-05), (1392, 1, 2.3685457129322596e-05), (487, 1, 2.3685457129322596e-05), (10885, 1, 3.158060950576346e-05), (2271, 1, 1.8948365703458078e-05), (2764, 1, 1.579030475288173e-05), (8704, 1, 1.1842728564661298e-05), (3755, 1, 1.1842728564661298e-05), (2720, 1, 2.3685457129322596e-05), (6485, 1, 9.474182851729039e-06), (10484, 1, 4.737091425864519e-05), (9001, 1, 2.3685457129322596e-05), (10356, 1, 1.3534546931041483e-05), (1493, 1, 3.158060950576346e-05), (8851, 1, 1.8948365703458078e-05), (5005, 1, 1.1842728564661298e-05), (4655, 1, 3.158060950576346e-05), (5526, 1, 1.1842728564661298e-05), (1131, 1, 1.579030475288

In [0]:
import nltk
nltk.download('stopwords')
from bs4 import BeautifulSoup
import collections
from collections import Counter
import re
nltk.download('punkt')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


<pre>Result ranking using TF-IDF and cosine similarity measure</pre>

In [0]:
import ipywidgets as widgets
from IPython.display import display
%xmode Verbose
import math
import itertools
import operator

def onclick(event):
  with result:
    new(keyword.value)

def new(query):
  keyword = query
  #cleaning our search  query

  keyword = str(keyword).lower() #convert into lowercase
  cleantext = BeautifulSoup(keyword).get_text()

  cleantext = re.sub("\d+", " ", cleantext) #remove digits
  cleantext = re.sub('[^A-Za-z0-9]+', ' ', cleantext) #remove special characters
  cleantext = re.sub("\s+", " ", cleantext) #remove extra spaces

  cleantext = word_tokenize(cleantext)
  cleantext = [word for word in cleantext if word not in stopwords.words('english')]
  counter = Counter(cleantext)
  query = dict()
  maxf = Counter(cleantext).most_common(1)[0][1]
  #maxf of our search query

  # (word, [(freq,tfidf), ...])
  docfreq = 0
  for ele in counter:
    tf = counter[ele]/maxf
    query[ele] = tf
    docfreq = docfreq + (tf * tf)
  docfreq = math.sqrt(docfreq)
  # print(docfreq)
  # print(query)
  sql="select * from postings where word in ({seq})".format(
    seq=','.join(['?']*len(cleantext)))

  c.execute(sql, cleantext)
  result = c.fetchall()
  # print(result)
  # (word, [(did,freq,tfidf), ...])
  # (did, (maxf,magnitude squared))
  all_docs_arr = dict()
  for row in result:
    docid = re.findall(r'\((.*?),.*?[+-]?([0-9]*[.]?[0-9]+)\)', row[1])
    for row_1 in docid:
      all_docs_arr[str(row_1[0])] = {}
      all_docs_arr[str(row_1[0])][str(row[0])] = row_1[1]
  # print(all_docs_arr)
  docids_unique = list(all_docs_arr.keys())


  docs_maxf = {}
  for vals in docids_unique:
    sql_look = """ SELECT * FROM docmag where docid = ? """
    c.execute(sql_look, [vals])
    result_docs = c.fetchall()
    # print(result_docs)
    for val in result_docs:
      docs_maxf[str(val[0])] = val[2]
  # print(docs_maxf)

  cosine_similarity = {}
  for docid in docids_unique:
    numerator = 0
    for text in cleantext:
      try:
        tfifd_doc = float(all_docs_arr[docid][text])
        tf_query  = float(query[text])
        numerator = numerator + (tfifd_doc*tf_query)
        denominator = docs_maxf.get(docid)*docfreq
      except:
        break
    try:
      denominator = docs_maxf.get(docid)*docfreq
      cosine_similarity[docid] = (numerator/denominator)
    except:
      break
  highest_first = sorted(cosine_similarity.items(),key=operator.itemgetter(1),reverse=False)
  print("Results fetched in docids from highest similar to lowest")
  for rows in highest_first:
    print(rows[0])  

Exception reporting mode: Verbose


<pre>Interface for Keyword search functionality and displaying results</pre>

In [24]:
import ipywidgets as widgets
from IPython.display import display

keyword = widgets.Text(placeholder= 'Enter search query')
search_button = widgets.Button(description='Search')
result = widgets.Output()

display(keyword,search_button,result)

search_button.on_click(onclick)

Text(value='', placeholder='Enter search query')

Button(description='Search', style=ButtonStyle())

Output()